In [1]:
'''
Coding: PYTHON UTF-8
Created On: 2023-07-05 12:50:43
Author: Putu Hendra Widyadharma
=== filter .pha using wadati result from makeWADATI 
=== filter based on Travel Time vs Distance Data
'''

'\nCoding: PYTHON UTF-8\nCreated On: 2023-07-05 12:50:43\nAuthor: Putu Hendra Widyadharma\n=== filter .pha using wadati result from makeWADATI \n=== filter based on Travel Time vs Distance Data\n'

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from localfunction import *
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [3]:
path = 'G:\\My Drive\\IUGG Seminar\\data\\data_poster\\'
fname = 'arrivals.dat'
fname2 = 'temppha_arrivals.txt' #temppha
fname3 = 'tempwadati_arrivals.txt' #tempwadati
deviation_p = 3
deviation_s = 3

In [4]:
df = readabsolute(path+fname)
dfphase = pd.read_csv(path+fname2,delim_whitespace=True)
dfwadati = pd.read_csv(path+fname3,delim_whitespace=True)

In [5]:
# cek agar not na
dfwadati = dfwadati[~dfwadati['dist_from_event'].isna()]

In [6]:
# hitung dan buat persamaan pembatas 
slopep, interceptp = np.polyfit(dfwadati['dist_from_event'],dfwadati['tp'],1)
slopes, intercepts = np.polyfit(dfwadati['dist_from_event'],dfwadati['ts'],1)

# manual slope
# slope = 0.589
# intercept = 0.9170

# predict p function
predictp = np.poly1d([slopep, interceptp])
# predict with deviation
predictpup = np.poly1d([slopep,interceptp+deviation_p])
predictpdown = np.poly1d([slopep,interceptp-deviation_p])

# predict s function
predicts = np.poly1d([slopes, intercepts])
# predict with deviation
predictsup = np.poly1d([slopes,intercepts+deviation_s])
predictsdown = np.poly1d([slopes,intercepts-deviation_s])


# hitung R^2 menguji kecocokan regresi
r2p = r2_score(dfwadati['dist_from_event'],predictp(dfwadati['tp']))
r2s = r2_score(dfwadati['dist_from_event'],predicts(dfwadati['ts']))

print("p-phase")
print('VP/VS = ' + str(slopep + 1))
print(f"R-square = {r2p}\n")

print("s-phase")
print('VP/VS = ' + str(slopes + 1))
print(f"R-square = {r2s}")

p-phase
VP/VS = 1.1669057729743837
R-square = -0.6888817812947248

s-phase
VP/VS = 1.2709680122602707
R-square = -0.4027836972287102


In [7]:
# =============================================================================
# buat grafik travel time vs distance bubble plot
# =============================================================================
plt.rcParams.update({'font.size': 14})
from matplotlib import colors
from matplotlib.colors import Colormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

x = input('p,s,both')

#plot
fig, ax = plt.subplots(figsize = (5,5), dpi=1200)
if x == 'p' or x == 'both':
    ax.scatter(dfwadati['dist_from_event'],dfwadati['tp'],color='blue',label="P-phase",edgecolors='black',s=1)
    ax.plot(dfwadati['dist_from_event'],predictp(dfwadati['dist_from_event']),'r--')
    ax.plot(dfwadati['dist_from_event'],predictpup(dfwadati['dist_from_event']),'r--')
    ax.plot(dfwadati['dist_from_event'],predictpdown(dfwadati['dist_from_event']),'r--')

if x == 's' or x == 'both':
    ax.scatter(dfwadati['dist_from_event'],dfwadati['ts'],color='red',label="S-phase",edgecolors='black',s=1)
    ax.plot(dfwadati['dist_from_event'],predicts(dfwadati['dist_from_event']),'b--')
    ax.plot(dfwadati['dist_from_event'],predictsup(dfwadati['dist_from_event']),'b--')
    ax.plot(dfwadati['dist_from_event'],predictsdown(dfwadati['dist_from_event']),'b--')

ax.set_xlabel('Epicentral Distance (km)')
ax.set_ylabel('Travel Time (s)')
ax.set_xlim([-2,100])
ax.set_ylim([-2,40])
ax.legend()
fig.set_figheight(5)
fig.set_figwidth(10)
# fig.savefig(outputpath+'Time Travel Diagram Bubble {}.jpg'.format(Path(fname).stem))

In [17]:
### dfwadati processing
# fungsi seleksi data berdasarkan batas atas dan bawah deviasi travel time
def selection_data_p(row):
    if (
        (row['tp'] <= predictpup(row['dist_from_event'])) and (row['tp'] >= predictpdown(row['dist_from_event']))
       ):
        return True
    else:
        return False
    
def selection_data_s(row):
    if (
        (row['ts'] <= predictsup(row['dist_from_event'])) and (row['ts'] >= predictsdown(row['dist_from_event']))
       ):
        return True
    else:
        return False

# apply perhitungan
dfwadati['result_p'] = dfwadati.apply(selection_data_p,axis=1)
dfwadati['result_s'] = dfwadati.apply(selection_data_s,axis=1)
# select wadati hanya pada event yang perlu diubah
dfwadati = dfwadati[dfwadati['event_id'].isin(dfwadati[(dfwadati['result_p'] == False) | (dfwadati['result_s'] == False)]['event_id'].unique())]

In [20]:
# index search untuk event_id yang perlu disesuaikan menurut wadati 
editable_idx = df[df[0] == '#'][14].astype('int64').isin(dfwadati['event_id'].unique())
editable_idx = editable_idx[editable_idx==True].index
editable_idx

Int64Index([   0,   94, 2690, 2724, 3257, 3495, 3702, 3991, 4531, 5065, 5233,
            5295, 5343, 5369, 5500, 5530, 5565, 5589, 5627, 5675, 5711, 5730,
            5797, 5814, 5830, 5896, 6057, 6093, 6113, 6178, 6233, 6420, 6578,
            6692, 6779, 6853, 6895, 6994, 7023, 7071, 7145, 7185, 7219, 7238,
            7273, 7369, 7424, 7557, 7664, 7695, 7707, 7786, 7811, 7904, 7919,
            7963, 8003, 8085, 8100, 8135, 8196, 8209, 8277, 8310, 8325, 8366,
            8392, 8416, 8443, 8490, 8606, 8623, 8727, 8781, 8862, 8911, 8930,
            8994, 9051, 9141, 9237, 9279, 9332, 9522, 9537],
           dtype='int64')

In [21]:
# buat tempat data hasil
df_result = pd.DataFrame([],columns = df.columns)

# pha index iterable
idx = df[df[0] == '#'].index

for idx_ in range (len(idx)):
# for idx_ in [1099]:
# for idx_ in [0,1,2,3]:
    # buat data per kejadian gempa
    if idx_ == len(idx)-1:
        tempdf = df.iloc[idx[idx_]::]
    else:
        tempdf = df.iloc[idx[idx_]:idx[idx_+1],:]
    # cek index header terhadap editable_idx
    if tempdf.index[0] in editable_idx:
        # proses delete data fasa S stasiun yang ts-tp diluar batas deviasi
        for i, row_ in tempdf.iterrows():
            # cari fasa P dan list stasiun yang masuk dalam membuat tp vs dist buruk 
            if row_[3]=='P' and row_[0] in (dfwadati[(dfwadati['event_id'] == int(tempdf.iloc[0,14])) & (dfwadati['result_p'] == False)]['sta'].to_list()):
                print(f'event: {tempdf.iloc[0,14]} -- index:{row_.name} --> {row_[0]} Phase:{row_[3]} dropped')
                tempdf.drop(row_.name,inplace=True)
                
            # cari fasa S dan list stasiun yang masuk dalam membuat ts vs dist buruk 
            if row_[3]=='S' and row_[0] in (dfwadati[(dfwadati['event_id'] == int(tempdf.iloc[0,14])) & (dfwadati['result_s'] == False)]['sta'].to_list()):
                print(f'event: {tempdf.iloc[0,14]} -- index:{row_.name} --> {row_[0]} Phase:{row_[3]} dropped')
                tempdf.drop(row_.name,inplace=True)
    else:
        pass
    # masukkan hasil ke df hasil
    df_result = pd.concat([df_result,tempdf])

event: 1 -- index:12 --> CBJI Phase:S dropped
event: 7 -- index:97 --> BM95 Phase:P dropped
event: 7 -- index:107 --> SCJM Phase:S dropped
event: 135 -- index:2691 --> BM92 Phase:P dropped
event: 135 -- index:2692 --> BM92 Phase:S dropped
event: 137 -- index:2747 --> CWJM Phase:S dropped
event: 167 -- index:3259 --> BM60 Phase:S dropped
event: 181 -- index:3497 --> BM63 Phase:S dropped
event: 194 -- index:3706 --> BM63 Phase:S dropped
event: 209 -- index:3993 --> BM63 Phase:S dropped
event: 237 -- index:4534 --> BM95 Phase:S dropped
event: 237 -- index:4536 --> BM63 Phase:S dropped
event: 237 -- index:4540 --> BM94 Phase:S dropped
event: 266 -- index:5068 --> BM60 Phase:S dropped
event: 274 -- index:5245 --> SCJM Phase:S dropped
event: 277 -- index:5314 --> CWJM Phase:P dropped
event: 277 -- index:5315 --> CWJM Phase:S dropped
event: 277 -- index:5319 --> PSJM Phase:S dropped
event: 277 -- index:5321 --> WLJI Phase:P dropped
event: 277 -- index:5322 --> WLJI Phase:S dropped
event: 279 

In [22]:
# perbaiki df_result dan simpan dalam bentuk pha
df_result.sort_index(inplace = True)
df_result.reset_index(inplace = True, drop = True)
df2dat(df_result,evnum = 1, path = path, fname=Path(fname).stem+'_ttdistfilter.dat')
print("== data filter")
readeventphase(path+Path(fname).stem+'_ttdistfilter.dat')

== data filter
! arrivals_ttdistfilter.dat
---------------------------
start time        : 2022-11-23 07:10:15.290000
end time          : 2022-12-03 00:13:23.900000
minimum magnitude : 0.38
maximum magnitude : 4.39
minimum depth     : 0 km
maximum depth     : 25 km
station used      : 44
---------------------------
Event :    484
Phase :   8974
-- P  :   5836
-- S  :   3138
---------------------------
